# Citybike Standorte & Nutzungsverhalten in Wien

In diesem Jupyter Notebook zeigen wir, wie man Wetterdaten und Nextbike-Nutzungsdaten für Wien in Echtzeit scrapt, an Kafka sendet und mit Kafka-Consumer empfängt, um sie anschließend grafisch darzustellen. Dies umfasst das Scrapen von Wetterdaten von einer Webseite, das Abrufen von Nextbike-Daten über eine API, das Senden der Daten an Kafka-Topics und die Visualisierung der Daten als interaktive App und in Graphen. 

### Datasources:

- API von Nextbike zur Ermittlung der Nextbike Standorte und Auslastung
  https://api.nextbike.net/maps/nextbike-live.json
- Webscraping zum Scrapen der (historischen) Wetterdaten für Wien
  https://www.wunderground.com/history/daily/at/vienna
- POIs (Point of Interest) in Wien
  https://www.data.gv.at/katalog/dataset/f4e80988-c139-4953-8176-b3d6d03f6449

### Architecture

![Alt Text](Slide2.PNG)

github: https://github.com/aoezalti/BDENG

## Install libraries & packages

In [ ]:
## Install libraries & install packages needed to run MongoDB
import requests
import time
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
import pandas as pd
import seaborn as sns
os.environ['PATH'] += os.pathsep + '/usr/local/bin' 

## Start Docker container

In [ ]:
!docker-compose up -d

### Check if the services are running

In [ ]:
!docker ps

# Process CSV Data

### Read data and display the first few rows

In [ ]:
import pandas as pd

# Read the CSV file with the specified encoding
poi_df = pd.read_csv('data/top-locations-wien.csv', encoding='latin1', sep=';')


# Display the first few rows of the dataframe
print(poi_df.head())

### Create new DataFrame for the Vizualisation (category and coordinates)

In [ ]:
# Auswahl der gewünschten Spalten
new_df = poi_df[['category', 'geo_latitude', 'geo_longitude']]
# Ausgabe des neuen DataFrames
print(new_df)

### Check for missing values

In [ ]:
print(new_df.isnull().sum())

## CSV-Data cleaning

In [ ]:
# Fill or drop missing values
new_df = new_df.dropna()  # Dropping rows with missing values

# Check for duplicates
print(new_df.duplicated().sum())

# Remove duplicates
new_df = new_df.drop_duplicates()

### Show clean Dataframe

In [ ]:
print(new_df)

## Save data locally

In [ ]:
# Speichern des DataFrames als CSV-Datei mit richtiger Spaltenstruktur
new_df.to_csv('data/poi_coordinates.csv', index=False, sep=';')

print("Datei wurde erfolgreich im 'data'-Ordner gespeichert.")

## Analyse and Vizualisation of CSV-Data

In [ ]:
!pip install pandas matplotlib seaborn folium geopandas shapely

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point

# Gegebene Daten für Kategorien und Koordinaten
data = pd.read_csv('data/poi_coordinates.csv', encoding='latin1', sep=';')

# DataFrame erstellen
new_df = pd.DataFrame(data)

# Filtern nach den gewünschten Kategorien
desired_categories = ['museum', 'shopping', 'cafes', 'musicstage', 
                      'restaurants', 'sightseeing', 'nightlife', 
                      'gastronomy', 'accommodations']
new_df['category'] = new_df['category'].str.lower()  # Konvertierung zu Kleinbuchstaben für die Kategorienvergleich
new_df = new_df[new_df['category'].isin(desired_categories)]

# Konvertierung der Koordinaten von Strings zu Floats
new_df['geo_latitude'] = new_df['geo_latitude'].str.replace(',', '.').astype(float)
new_df['geo_longitude'] = new_df['geo_longitude'].str.replace(',', '.').astype(float)

# Grundlegende Analyse
print(new_df.describe())
print(new_df['category'].value_counts())

# Visualisierung der Verteilung der Kategorien mit Matplotlib und Seaborn
plt.figure(figsize=(10, 6))
sns.countplot(data=new_df, x='category', palette='viridis')
plt.title('Verteilung der Kategorien')
plt.xlabel('Kategorie')
plt.ylabel('Anzahl')
plt.show()

# Erstellen einer Karte für Wien mit Folium
map_vienna = folium.Map(location=[48.210033, 16.363449], zoom_start=13)

# MarkerCluster für eine bessere Darstellung von dichten Markergruppen
marker_cluster = MarkerCluster().add_to(map_vienna)

# Funktion zur Zuordnung von Farben basierend auf der Kategorie
def get_color(category):
    if category == 'museum':
        return 'purple'
    elif category == 'shopping':
        return 'blue'
    elif category == 'cafes':
        return 'green'
    elif category == 'musicstage':
        return 'orange'
    elif category == 'restaurants':
        return 'red'
    elif category == 'sightseeing':
        return 'darkblue'
    elif category == 'nightlife':
        return 'darkred'
    elif category == 'gastronomy':
        return 'lightgreen'
    elif category == 'accommodations':
        return 'cadetblue'
    else:
        return 'gray'  # Fallback für unbekannte Kategorien

# Hinzufügen der POI-Marker zur Karte
for _, row in new_df.iterrows():
    folium.Marker(
        location=[row['geo_latitude'], row['geo_longitude']],
        popup=row['category'].capitalize(),
        icon=folium.Icon(color=get_color(row['category']))
    ).add_to(marker_cluster)

# Speichern der Karte als HTML-Datei
map_vienna.save('data/poi_map_vienna1.html')

# Ausgabe, um den Benutzer darüber zu informieren, dass die Karte gespeichert wurde
print("Die Karte wurde erfolgreich als 'poi_map_vienna.html' gespeichert.")


# Kafka Producer & Consumer

In [ ]:
# Load important libraries
from kafka import KafkaProducer
import requests
import json
import time
# Create Kafka producer
producer = KafkaProducer(bootstrap_servers=['localhost:29093'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [ ]:
import pandas as pd

# Gegebene Daten für Kategorien und Koordinaten
data = pd.read_csv('data/poi_coordinates.csv', encoding='latin1', sep=';')

# DataFrame erstellen
new_df = pd.DataFrame(data)

# Filtern nach den gewünschten Kategorien
desired_categories = ['museum', 'shopping', 'cafes', 'musicstage', 
                      'restaurants', 'sightseeing', 'nightlife', 
                      'gastronomy', 'accommodations']
new_df['category'] = new_df['category'].str.lower()  # Konvertierung zu Kleinbuchstaben für die Kategorienvergleich
new_df = new_df[new_df['category'].isin(desired_categories)]

# Bereinigung der Koordinaten
new_df['geo_latitude'] = new_df['geo_latitude'].str.replace(',', '.')
new_df['geo_longitude'] = new_df['geo_longitude'].str.replace(',', '.')

# Entfernen von Zeilen mit ungültigen Koordinaten
new_df = new_df[new_df['geo_latitude'].apply(lambda x: x.replace('.', '', 1).isdigit())]
new_df = new_df[new_df['geo_longitude'].apply(lambda x: x.replace('.', '', 1).isdigit())]

# Konvertieren der Koordinaten in das richtige Format
new_df['geo_latitude'] = new_df['geo_latitude'].astype(float)
new_df['geo_longitude'] = new_df['geo_longitude'].astype(float)

# Debug-Ausgabe nach der Konvertierung
print("Nach der Konvertierung:")
print(new_df[['geo_latitude', 'geo_longitude']].head())
print(new_df.dtypes)


In [ ]:
import requests
print(new_df.head())
print(new_df.dtypes)
# REST API URL
api_url = 'https://api.nextbike.net/maps/nextbike-live.json'

def fetch_data():
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Fehler beim Abrufen der Daten: {e}")
        return None

nextbike_data = fetch_data()

# Beispielstruktur: Daten verarbeiten
wien_data = []

if nextbike_data is not None:
    for country in nextbike_data.get('countries', []):
        if country.get('country_name') == 'Austria':
            for city in country.get('cities', []):
                if city.get('name') == 'Wien':
                    for place in city.get('places', []):
                         if not place.get('name', '').startswith('BIKE'):
                            station_data = {
                                'station': place.get('name'),
                                'geo_latitude': str(place.get('lat')).replace(',', '.'),
                                'geo_longitude': str(place.get('lng')).replace(',', '.'),
                                'available_bikes': place.get('bikes')
                            }
                            wien_data.append(station_data)

# Umwandlung in ein Pandas DataFrame
wien_df = pd.DataFrame(wien_data)
wien_df['geo_latitude'] = wien_df['geo_latitude'].astype(float)
wien_df['geo_longitude'] = wien_df['geo_longitude'].astype(float)

# Entfernen der ersten Zeile
wien_df = wien_df.drop(index=0).reset_index(drop=True)

# dataframe als csv speichern
csv_file_path = 'data/wien_data.csv'
wien_df.to_csv(csv_file_path, index=False)

# Überprüfen der Spaltennamen und Daten
print("Wien DataFrame:")
print(wien_df.head())
print(wien_df.dtypes)


# Set up Spark

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
import os

# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaSparkStream") \
    .master("spark://172.29.16.102:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "1") \
    .getOrCreate()

In [ ]:
# Konvertiere Pandas DataFrame in Spark RDD
wien_rdd = sc.parallelize(wien_df.to_dict(orient='records'))

print("Inhalt des Wien RDD:")
for record in wien_rdd.take(5):
    print(record)

# Zählen der Anzahl der Stationen
station_count = wien_rdd.count()
print(f"Anzahl der Stationen: {station_count}")

# durchschnittliche Anzahl der verfügbaren Fahrräder pro Station
average_bikes = wien_rdd.map(lambda x: x['available_bikes']).mean()
print(f"Durchschnittliche Anzahl der verfügbaren Fahrräder pro Station: {average_bikes}")

# Verteilung der verfügbaren Fahrräder pro Station
bikes_distribution = wien_rdd.map(lambda x: (x['station'], x['available_bikes'])).collect()
print("Verteilung der verfügbaren Fahrräder pro Station:")
for station, bikes in bikes_distribution:
    print(f"Station: {station}, Verfügbare Fahrräder: {bikes}")


### Experimentieren mit Spark RDDs

In [ ]:
# Beispiel: CSV-Datei laden
file_path = "data/poi_coordinates.csv"  # Pfad zu Ihrer CSV-Datei
rdd = spark.read.csv(file_path, header=True, inferSchema=True)

header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header)

# Daten aufteilen und analysieren
data_rdd = data_rdd.map(lambda row: row.split(','))  # Annahme: CSV ist komma-separiert

In [ ]:
# Beispielanalyse: Zählen der Anzahl von Einträgen pro Kategorie
category_counts = data_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)

# Ergebnisse anzeigen
print("Anzahl der Einträge pro Kategorie:")
for category, count in category_counts.collect():
    print(f"{category}: {count}")

In [ ]:
# Kafka-Parameter
kafka_bootstrap_servers = 'localhost:29093'
kafka_topic = 'nextbike_wien'

# Definieren des Schemas für die JSON-Daten
schema = StructType([
    StructField("station", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("available_bikes", IntegerType(), True)
])

# Daten von Kafka lesen
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

df = df.selectExpr("CAST(value AS STRING)")
df = df.withColumn("jsonData", from_json(col("value"), schema)).select("jsonData.*")

# Streaming-Daten in der Konsole anzeigen
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

In [ ]:
spark.stop()

# Karte mit POIs und Nextbike-Stationen

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Erstellen einer Karte für Wien mit Folium
map_vienna = folium.Map(location=[48.210033, 16.363449], zoom_start=13)

# MarkerCluster für eine bessere Darstellung von dichten Markergruppen
poi_marker_cluster = MarkerCluster().add_to(map_vienna)
nextbike_marker_cluster = MarkerCluster().add_to(map_vienna)

# Funktion zur Zuordnung von Farben basierend auf der Kategorie
def get_color(category):
    if category == 'museum':
        return 'purple'
    elif category == 'shopping':
        return 'blue'
    elif category == 'cafes':
        return 'green'
    elif category == 'musicstage':
        return 'orange'
    elif category == 'restaurants':
        return 'red'
    elif category == 'sightseeing':
        return 'darkblue'
    elif category == 'nightlife':
        return 'darkred'
    elif category == 'gastronomy':
        return 'lightgreen'
    elif category == 'accommodations':
        return 'cadetblue'
    else:
        return 'gray'  # Fallback für unbekannte Kategorien

# POI-Marker
for _, row in new_df.iterrows():
    folium.Marker(
        location=[row['geo_latitude'], row['geo_longitude']],
        popup=row['category'].capitalize(),
        icon=folium.Icon(color=get_color(row['category']))
    ).add_to(poi_marker_cluster)

# Nextbike-Stationen
for _, row in wien_df.iterrows():
    folium.Marker(
        location=[row['geo_latitude'], row['geo_longitude']],
        popup=f"Station: {row['station']}, Available bikes: {row['available_bikes']}",
        icon=folium.Icon(color='pink', icon='bicycle', prefix='fa')  # Verwenden Sie ein anderes Symbol und eine andere Farbe
    ).add_to(nextbike_marker_cluster)

# Speichern der Karte als HTML-Datei
map_vienna.save('data/poi_map_vienna2.html')

print("Die Karte wurde erfolgreich als 'poi_map_vienna2.html' gespeichert.")

# Zuordnung bei welchen Kategorien die meisten Nextbike-Stationen sind 

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Konvertieren der Koordinaten in floats
poi_df['geo_latitude'] = poi_df['geo_latitude'].str.replace(',', '.').astype(float)
poi_df['geo_longitude'] = poi_df['geo_longitude'].str.replace(',', '.').astype(float)

# Erstellen von GeoDataFrames
poi_gdf = gpd.GeoDataFrame(poi_df, geometry=gpd.points_from_xy(poi_df.geo_longitude, poi_df.geo_latitude))

print("POI GeoDataFrame nach der Konvertierung:")
print(poi_gdf[['geo_latitude', 'geo_longitude', 'category']].head())
print(poi_gdf.dtypes)


In [ ]:
# Konvertieren der Koordinaten in das richtige Format
wien_df['geo_latitude'] = wien_df['geo_latitude'].astype(float)
wien_df['geo_longitude'] = wien_df['geo_longitude'].astype(float)

# Erstellen von GeoDataFrames für Nextbike-Stationen
wien_gdf = gpd.GeoDataFrame(wien_df, geometry=gpd.points_from_xy(wien_df.geo_longitude, wien_df.geo_latitude))

print(wien_gdf[['geo_latitude', 'geo_longitude', 'station']].head())
print(wien_gdf.dtypes)

In [ ]:
poi_gdf.set_crs(epsg=4326, inplace=True, allow_override=True)
wien_gdf.set_crs(epsg=4326, inplace=True, allow_override=True)

buffer_radius = 1500  # 1500 Meter

# Projektion in ein metrisches System (z.B. UTM) für das Erstellen der Puffer
poi_gdf = poi_gdf.to_crs(epsg=32633)  # UTM Zone 33N für Wien
wien_gdf = wien_gdf.to_crs(epsg=32633)

# Erstellen der Puffer um die POIs
poi_gdf['buffer'] = poi_gdf.geometry.buffer(buffer_radius)

poi_gdf.set_geometry('buffer', inplace=True)

joined_gdf = gpd.sjoin(wien_gdf, poi_gdf[['category', 'buffer']], predicate='within')

# Zähle die Anzahl der Nextbike-Stationen in der Nähe jeder POI-Kategorie
category_station_counts = joined_gdf.groupby('category').size().reset_index(name='station_count')

print("Anzahl der Nextbike-Stationen in der Nähe jeder POI-Kategorie:")
print(category_station_counts)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualisieren der Anzahl der Nextbike-Stationen in der Nähe jeder POI-Kategorie
plt.figure(figsize=(12, 6))
sns.barplot(data=category_station_counts, x='category', y='station_count', palette='viridis')
plt.title('Anzahl der Nextbike-Stationen in der Nähe jeder POI-Kategorie')
plt.xlabel('Kategorie')
plt.ylabel('Anzahl der Stationen')
plt.xticks(rotation=45)
plt.show()

# Experiment: Wetterdaten & Bike Nutzung plotten

In [ ]:
from kafka import KafkaConsumer
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

# Kafka Consumer für Wetterdaten
weather_consumer = KafkaConsumer(
    'weather-data',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='weather-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

# Kafka Consumer für Nextbike-Daten
nextbike_consumer = KafkaConsumer(
    'nextbike_wien',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='nextbike-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

# DataFrames zum Speichern der empfangenen Daten
weather_data = pd.DataFrame(columns=['Zeit', 'Temperatur'])
nextbike_data = pd.DataFrame(columns=['station', 'geo_latitude', 'geo_longitude', 'available_bikes'])

# Funktion zum Aktualisieren der Plots
def update_plots():
    global weather_data, nextbike_data
    
    plt.figure(figsize=(14, 7))

    # Plot für die Temperaturdaten
    plt.subplot(2, 1, 1)
    sns.lineplot(data=weather_data, x='Zeit', y='Temperatur')
    plt.title('Temperaturverlauf')
    plt.xlabel('Zeit')
    plt.ylabel('Temperatur (°C)')

    # Plot für die Nextbike-Nutzung
    plt.subplot(2, 1, 2)
    sns.lineplot(data=nextbike_data, x='Zeit', y='available_bikes')
    plt.title('Nextbike-Nutzung')
    plt.xlabel('Zeit')
    plt.ylabel('Verfügbare Fahrräder')

    plt.tight_layout()
    plt.show()

# Funktion zum Empfangen der Wetterdaten
def consume_weather_data():
    global weather_data
    for message in weather_consumer:
        data = message.value
        data['Zeit'] = datetime.strptime(data['Zeit'], '%H:%M')
        data['Temperatur'] = float(data['Temperatur'].replace('°C', '').strip())
        weather_data = weather_data.append(data, ignore_index=True)
        if len(weather_data) > 500:
            weather_data = weather_data[-500:]
        update_plots()

# Funktion zum Empfangen der Nextbike-Daten
def consume_nextbike_data():
    global nextbike_data
    for message in nextbike_consumer:
        data = message.value
        data['Zeit'] = datetime.now()
        nextbike_data = nextbike_data.append(data, ignore_index=True)
        if len(nextbike_data) > 500: 
            nextbike_data = nextbike_data[-500:]
        update_plots()

# Starten der Consumer in separaten Threads
from threading import Thread

weather_thread = Thread(target=consume_weather_data)
nextbike_thread = Thread(target=consume_nextbike_data)

weather_thread.start()
nextbike_thread.start()

weather_thread.join()
nextbike_thread.join()